In [1]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import f1_score

In [2]:
with open('images_features_2d.pickle','rb') as features:

    data=pickle.load(features)

In [3]:
data['CXR1_1_IM-0001-3001.png'].shape

(1, 7, 7, 1024)

In [4]:
def fetch_tags(image_id, tags):

    new_tags=[]

    for image in image_id.to_list():

        tag=tags[image.split('_')[0]]

        if tag[0].lower()=='normal':
            new_tags.append(0)
        else:
            new_tags.append(1)

    return new_tags

path='/media/zaheer/Data/Image_Text_Datasets/IU_Xray/latest/One_Image_norm_abnorm_split/word/Sample1/'
def load_preprocess_data():
    tags = np.load('Data_with_tags.npy', allow_pickle=True).item()
    train = pd.read_json(path+'/train/train.json')
    #print(train.shape, train.columns)
    x_train=train['image_id']

    train_tags=fetch_tags(x_train,tags)
    #print(train_tags)
    #y_train = mlb.fit_transform([train_tags])
    y_train=pd.Series(train_tags)
    #print(y_train)

    #print(y_train.head())

    test = pd.read_json(path+'/test/test.json')
    x_test = test['image_id']

    test_tags=fetch_tags(x_test,tags)
    y_test = pd.Series(test_tags)
    
    return x_train,y_train,x_test,y_test



In [5]:
def load_features(split):
    features = []#np.empty((0,7,7,1024))
    
    for image in split.tolist():
        #print(image)
        #features=np.hstack((features, data[image]))
        #features=np.append(features, data[image], axis=0)
        features.append(data[image])
    features=np.array(features)
    features=np.reshape(features,(-1,49,1024))
    
    return features

In [6]:
def class_model(n):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(49,1024)))
    
    model.add(layers.GlobalAveragePooling1D())
    #model.add(layers.Dense(128,activation='relu'))
    #model.add(layers.LeakyReLU(alpha=0.05))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(n, activation="sigmoid"))

    model.summary()
    
    return model



In [7]:
images_train,y_train,images_test,y_test=load_preprocess_data()

ValueError: Expected object or value

In [ ]:
y_train

In [ ]:
train_features=load_features(images_train)
test_features=load_features(images_test)

In [ ]:
model=class_model(1)

In [ ]:
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, verbose=0, mode='min')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
])

model.fit(train_features, y_train.to_numpy(), epochs=50, callbacks=[earlyStopping],validation_data=(test_features, y_test.to_numpy()), batch_size=16)

test_tags=model.predict(test_features)

In [ ]:
test_tags=np.where(test_tags<0.60,0,1)
test_tags

In [ ]:
test_tags=[i[0] for i in test_tags.tolist()]
test_tags=pd.Series(test_tags)

In [ ]:
y_test

In [ ]:
print(f1_score(y_test,test_tags))
#print(y_test)

In [ ]:
m=tf.keras.metrics.Recall()
m.update_state(y_test,test_tags)
m.result().numpy()

In [ ]:
import pickle

all_tags={}

for idx,image in enumerate(images_test.to_list()):
    all_tags[image.split('_')[0]]=test_tags[idx]
    
train_true=np.array(y_train)
for idx,image in enumerate(images_train.to_list()):
    all_tags[image.split('_')[0]]=train_true[idx]

print(all_tags['CXR63'])
with open(path+'binary_tags_chex.pkl','wb') as file:
    pickle.dump(all_tags, file, protocol=2)
    